In [27]:
import sys
sys.path.append('../')

from autogpt.config import Config
from autogpt.commands.command import command
import json
import requests
import yaml

from duckduckgo_search import ddg


CFG = Config()

## NAVER MAP API
https://developers.naver.com/docs/serviceapi/search/movie/movie.md#%EC%98%81%ED%99%94

In [29]:
import json
import urllib
from urllib.request import Request, urlopen

def get_lng_lat_by_address(address):
    client_id = CFG.naver_api_id
    client_secret = CFG.naver_api_client_secret 
    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(address)
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200):
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        print(response_body)

        if response_body['meta']['totalCount'] == 1 : 
            lng = response_body['addresses'][0]['x']
            lat = response_body['addresses'][0]['y']
            
            return (lng, lat)
        else:
            print('address does not exist')
        
    else:
        print(f'address: {address} Error Code: {res}')


In [ ]:
start = '파도살롱'
goal = '안목살롱'
waypoint = ''

start = get_lng_lat_by_location(start)
end = get_lng_lat_by_location(goal)
waypoint = get_lng_lat_by_location(waypoint)

In [30]:
get_lng_lat_by_location("강남구")

{'status': 'OK', 'meta': {'totalCount': 1, 'page': 1, 'count': 1}, 'addresses': [{'roadAddress': '서울특별시 강남구', 'jibunAddress': '서울특별시 강남구', 'englishAddress': 'Gangnam-gu, Seoul, Republic of Korea', 'addressElements': [{'types': ['SIDO'], 'longName': '서울특별시', 'shortName': '서울특별시', 'code': ''}, {'types': ['SIGUGUN'], 'longName': '강남구', 'shortName': '강남구', 'code': ''}, {'types': ['DONGMYUN'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['ROAD_NAME'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['BUILDING_NUMBER'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['BUILDING_NAME'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['LAND_NUMBER'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['POSTAL_CODE'], 'longName': '', 'shortName': '', 'code': ''}], 'x': '127.047502', 'y': '37.517305', 'distance': 0.0}], 'errorMessage': ''}


('127.047502', '37.517305')

In [34]:
def get_optimal_route(start, goal, waypoints=None, option=''):
    assert start, 'start must be a tuple of (lng, lat)'
    assert goal, 'goal must be a tuple of (lng, lat)'

    assert len(start) == 2, 'start must be a tuple of (lng, lat)'
    assert len(goal) == 2, 'goal must be a tuple of (lng, lat)'

    waypoints = '|'.join([f"{point[0]},{point[1]}" for point in waypoints]) if waypoints else ''
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&waypoints={waypoints}&option={option}"

    client_id = CFG.naver_api_id
    client_secret = CFG.naver_api_client_secret
    
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if (rescode == 200):
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
    else:
        print(f'start: {start} goal: {goal}, waypoints: {waypoints}, Error Code: {rescode}')
        return None

get_optimal_route(start, goal)


AssertionError: start must be a tuple of (lng, lat)

In [36]:

import requests
import re

def find_nearby_restaurants(location):
    # 음식점 검색
    url = "https://openapi.naver.com/v1/search/local"

    client_id = CFG.naver_api_id
    client_secret = CFG.naver_api_client_secret
    
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": f"{location} 음식점",
        "display": 10,
        "start": 1,
        "sort": "random"
    }
    response = requests.get(url, headers=headers, params=params)
    result = response.json()

    restaurants = []
    for item in result['items']:
        name = item['title'].replace('<b>', '').replace('</b>', '')
        address = item['address']

        # 별점 추정
        url = "https://openapi.naver.com/v1/search/blog"
        params = {
            "query": f"{name} {address} 리뷰",
            "display": 5
        }
        response = requests.get(url, headers=headers, params=params)
        blog_result = response.json()

        ratings = []
        for blog in blog_result['items']:
            # 별점 정보가 포함된 문자열을 찾습니다 (예: 별점: 4.5, 평점: 3.0 등).
            match = re.search(r'별점\s*:\s*([\d.]+)|평점\s*:\s*([\d.]+)', blog['description'])
            if match:
                rating = match.group(1) or match.group(2)
                ratings.append(float(rating))

        # 별점 평균 계산
        if ratings:
            avg_rating = sum(ratings) / len(ratings)
            restaurant = f"{name} ({address}) - 별점: {avg_rating:.1f}"
        else:
            restaurant = f"{name} ({address}) - 별점 정보 없음"

        restaurants.append(restaurant)

    return restaurants


In [38]:
# 음식점 검색
url = "https://openapi.naver.com/v1/search/local"

client_id = CFG.naver_api_id
client_secret = CFG.naver_api_client_secret

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}
params = {
    "query": f"{location} 음식점",
    "display": 10,
    "start": 1,
    "sort": "random"
}
response = requests.get(url, headers=headers, params=params)
result = response.json()# 

In [39]:
result

{'errorMessage': 'NID AUTH Result Invalid (1000) : Authentication failed. (인증에 실패했습니다.)',
 'errorCode': '024'}

In [37]:
location = "서울역"
restaurants = find_nearby_restaurants(location)

for restaurant in restaurants:
    print(restaurant)

KeyError: 'items'

In [ ]:
url = "https://openapi.naver.com/v1/search/local"

client_id = CFG.naver_api_id
client_secret = CFG.naver_api_client_secret

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}
params = {
    "query": f"{location} 음식점",
    "display": 10,
    "start": 1,
    "sort": "random"
}
response = requests.get(url, headers=headers, params=params)
result = response.json()

restaurants = []
for item in result['items']:
    name = item['title'].replace('<b>', '').replace('</b>', '')
    address = item['address']

    # 별점 추정
    url = "https://openapi.naver.com/v1/search/blog"
    params = {
        "query": f"{name} {address} 리뷰",
        "display": 5
    }
    response = requests.get(url, headers=headers, params=params)
    blog_result = response.json()

    ratings = []
    for blog in blog_result['items']:
        # 별점 정보가 포함된 문자열을 찾습니다 (예: 별점: 4.5, 평점: 3.0 등).
        match = re.search(r'별점\s*:\s*([\d.]+)|평점\s*:\s*([\d.]+)', blog['description'])
        if match:
            rating = match.group(1) or match.group(2)
            ratings.append(float(rating))

    # 별점 평균 계산
    if ratings:
        avg_rating = sum(ratings) / len(ratings)
        restaurant = f"{name} ({address}) - 별점: {avg_rating:.1f}"
    else:
        restaurant = f"{name} ({address}) - 별점 정보 없음"

    restaurants.append(restaurant)